In [ ]:
# reload

%load_ext autoreload
%autoreload 2
from nn_functions import *

In [ ]:
model_save_num = 1 # start
samp_weights = None
plt.rcParams.update({'font.size': 22})
get_ipython().run_line_magic('precision', '3')

In [ ]:
# these values are not used anywhere, except for naming files to know what type of dataset we are working with 
# NOTE: ADD THIS INTO THE NAMING OF THE SAVED FILES ETC. 
time_step_size = '50ms'
past_sample_step_size = '10ms'



past_samples = 5

fill_na_val = 0

dev_list = ['wnc', 'sony'] # This comprises the train set 
delayPktSize_list = ['1400', '100']
#dev_list = ['sony'] # This comprises the train set 
#delayPktSize_list = ['100']
num_runs = 1 # Average over at least 10 runs

baseline_pred = 8.7 # This is the mean delay for 50 ms window size 

# Train on both but test on only one chipset 
test_on_one_chipset = False
test_on_chipset = 2 # 7 = sony, 2 = wnc

# Train on one chipset and test on the other 
transfer_bw_dev = False
train_on_chipset = 2 # 7 = sony, 2 = wnc

# Train and test on only the data from when the UEs were stationary  
use_only_stat_samples = False

# Train and test on only the data from when the UEs were moving  
use_only_moving_samples = False

# If True then we are predicting one window ahead if False then we are predicting on the same window 
shift_samp_for_predict = False

# In case I want to drop all the dl scheduling info and only focus on the ul and ABF features since the prediction is of ul delay and only ul is being loaded
drop_dl_cols = False

# Train on one dev 1400 B moving case and test on the same device 100 B pkt moving case 
# This way I do not have to shuffle the samples and can overlay the prediction and ground truth as it reached cell edge  
train_test_on_diff_pkt_sizes = False
# Make sure use_only_moving_samples = True so that only moving case samples used 

# If you want the test samples to be sorted by delay value to see the error differences for the low delay and high delay cases 
sort_test_samples = True

# All delay values above this will be removed from the train and test set
drop_outliers = True
delay_drop_th = 100 

In [ ]:
# If I want to do classification instead
# 7 classes. These show the upper limit
classification = False
delay_class_edges = [2, 5, 10, 20, 50, 100, 2000]
if classification:
    IN_PARAM = IN_PARAM_C
    OUT_PARAM = OUT_PARAM_C
    IN_PARAM['loss'] = 'dist_penalty'# 'class_mse' # default
else:
    IN_PARAM = IN_PARAM_R
    OUT_PARAM = OUT_PARAM_R
    IN_PARAM['loss'] = 'mse' # options are mse, mae and mape 
    IN_PARAM['eval_metric'] = 'mae' # options are mae, and mape
    
IN_PARAM['model_type'] = 'xgb'
IN_PARAM['model_save_num'] = model_save_num
IN_PARAM['time_wind_size'] = time_step_size
IN_PARAM['rand_seed'] = 10


In [ ]:
# take the top n features of each run and add it to the top_n_features list  
# If use_all_feats = True then thes will not be used 
feat_filter = 10 
top_n_features = []
use_all_feats = True 
selected_features = knowledge_based_features

if use_all_feats:
    #num_feat = 354 # keep nonGen features  
    #num_feat = 176 # drop dl features
    num_feat = 120*past_samples # default: drop nonGen features
    print('Num. of non-generalizable columns: ', len(expand_cols_to_step_size(non_gen_cols, past_samples)))
    # This is based on constant value columns that are constant over all cases and for both meas amd load UE.
    # It can't be done on a per file basis since it won't match the columns of other cases  
    print('Num. of constant value columns: ', len(expand_cols_to_step_size(const_val_cols, past_samples)))
    print('Total number of columns to be dropped here: ', len(expand_cols_to_step_size(drop_cols, past_samples)))
else:
    num_feat = len(expand_cols_to_step_size(selected_features, past_samples))


In [ ]:
# Take as input, class labels and then convert them to delay values by mapping them to 
# the mid-points of the bins. This way I can compute mae from class labels   
def class_mae(yhat, y, num_classes, delay_class_edges):
    # find the mid points of bins from the bin edges
    class_vals = (delay_class_edges + np.roll(delay_class_edges, 1))/2
    class_vals = class_vals[1:num_classes+1]
    # replace bin class labels with corresponding bin midpoints  
    yhat_val = [class_vals[int(i)] for i in yhat]
    y_val = [class_vals[int(i)] for i in y]
    # compute mae with these mid points
    return mean_absolute_error(yhat_val, y_val)

# Take as input class labels in continuous form and convert them to class labels 
# Then compute the confusion matrix 
def value_to_class_label(vals, delay_class_edges):
    delay_class_indices = np.digitize(vals, delay_class_edges)
    return (delay_class_indices - 1)



In [ ]:
run_seeds = range(0,num_runs)

if classification:
    avg_acc_over_runs = 0
    avg_prec_over_runs = 0
    avg_rec_over_runs = 0
    
    # I don't think this makes a lot of sense really 
    avg_err_over_runs = 0
    avg_baseline_err_over_runs = 0
else:
    avg_err_over_runs = 0
    avg_baseline_err_over_runs = 0
    # To aggregate error values per bin to see the error distribution for different ranges of the uplink delay values 
    n_bins = 30
    # Calculate the bin boundaries based on delay
    bin_edges = np.logspace(np.log10(1), np.log10(1600), n_bins+1)
    #print(bin_edges)

    train_bin_uldelay_mean = np.zeros(n_bins+1)
    train_bin_err_mean = np.zeros(n_bins+1)
    train_bin_perc_err_mean = np.zeros(n_bins+1)
    train_bin_baseline_err_mean = np.zeros(n_bins+1)
    train_bin_baseline_perc_err_mean = np.zeros(n_bins+1)
    train_bin_count = np.zeros(n_bins+1)

    test_bin_uldelay_mean = np.zeros(n_bins+1)
    test_bin_err_mean = np.zeros(n_bins+1)
    test_bin_perc_err_mean = np.zeros(n_bins+1)
    test_bin_baseline_err_mean = np.zeros(n_bins+1)
    test_bin_baseline_perc_err_mean = np.zeros(n_bins+1)
    test_bin_count = np.zeros(n_bins+1)

for r in run_seeds:
    print('Run with random seed: ', r, '------------------------------------------------------------------------')
    X = np.empty([0,num_feat])
    X[:] = np.nan
    y = np.empty([0,])
    y[:] = np.nan
    X_train = np.empty([0,num_feat])
    X_train[:] = np.nan
    y_train = np.empty([0,])
    y_train[:] = np.nan
    X_test = np.empty([0,num_feat])
    X_test[:] = np.nan
    y_test = np.empty([0,])
    y_test[:] = np.nan
    #feat_union = []
    for measUeDev in dev_list:
        print('Device type: ', measUeDev)
        for delayPktSize in delayPktSize_list:
            print('Delay pkt size', delayPktSize)
            IN_PARAM['rand_seed'] = r
            # Read the dataset
            op_col = 'owd_ul'
            if past_samples == 1:
                data = pd.read_csv('parsed_data/single_step_window/dataset_'+measUeDev+'_delayPktSize_'+delayPktSize+
                                       '_timeStepSize_'+time_step_size+'.csv', delimiter=",")
            else:
                data = pd.read_csv('parsed_data/5step_window/dataset_'+measUeDev+'_delayPktSize_'+delayPktSize+
                                       '_timeStepSize_'+time_step_size+'.csv', delimiter=",")
            print('Num. rows in data before dropping NAs due to owd_ul: ', data.shape[0])
            # Can't process samples where there is no ground truth
            data = data.dropna(subset=['owd_ul'])
            # don't know why I see these, but if I see any delay values of 0 then drop them
            data = data[data['owd_ul'] > 0]
            if drop_outliers:
                print('NOTE: dropping all rows with delay values > ', delay_drop_th)
                data = data[data['owd_ul'] <= delay_drop_th]
            
            if classification:
                # create class labels from the delay values
                data['owd_ul'] = value_to_class_label(data['owd_ul'], delay_class_edges)
                print('delay class distribution', data['owd_ul'].value_counts())
                num_classes = len(np.unique(data['owd_ul']))
                print('Number of classes: ', num_classes)
            
            # For the rest of the columns if there is an NA then just fill with 0 so that the rest of the metrics can still be used
            print('NOTE: filling NA in samples with ', fill_na_val)
            data = data.fillna(fill_na_val)
            
            # Why are there windows with NAs. This needs to be explained 
            print('Num. rows in data after dropping NAs due to owd_ul: ', data.shape[0])
            model_save_path = 'models/regressor_models/'
            print('debug1')
            # If needed isolate the rows that are for stationary UE before we drop the position column 
            if use_only_stat_samples:
                data = data[data['position'] != 0]
                print(data.shape)
            elif use_only_moving_samples:
                data = data[data['position'] == 0]
                print(data.shape)
        
            print('debug2')
            # Separate features and ground truth from the dataset 
            if use_all_feats:
                print('debug3')
                X_t = data.drop(expand_cols_to_step_size(drop_cols, past_samples), axis=1, errors='ignore')
                print('debug4')
                if drop_dl_cols:
                    assert past_samples == 1, "Including past samples can be done only with the uplink schedulinbg logs so it is not valid to use for drop_dl_cols = True"
                    X_t = X_t.drop([i for i in X_t.columns if 'dl_' in i], axis=1, errors='ignore')
                print(data.shape)
                print(X_t.shape)
                #print(set(feat_union) - set(X_t.columns))
                #feat_union = list(set(feat_union) | set( X_t.columns))
                #continue
            else:
                X_t = data[top_n_agg]
            
            X_feats = np.array(X_t.columns)
            print('debug5')
            # save into a file
            np.savetxt('input_feature_list.csv', X_feats, delimiter=',', fmt="%s")
            print('debug6')
            #print(list(set(X_t.columns)-set(X_feats)))
            #X_t = X_t.drop(list(set(X_t.columns)-set(X_feats)),axis=1)
            #print(X_feats)
            y_t = data[op_col]

            # convert everything to numpy 
            X_t = X_t.to_numpy()
            y_t = y_t.to_numpy()
            print('debug7')
            # plot the op_col histogram 
            #plt.figure(1)
            #plt.hist(y)
            #plt.show

            # Scale data
            #t = MinMaxScaler()
            #t.fit(X)
            #X = t.transform(X)

            if shift_samp_for_predict: 
                y_t = np.roll(y_t,1)
                X_t = X_t[1:]
                y_t = y_t[1:]
                
            X = np.append(X, X_t, axis=0)
            y = np.append(y, y_t, axis=0)
            print('debug8')
            #print(X.shape)
            #print(y.shape)
            
            if train_test_on_diff_pkt_sizes:
                if delayPktSize == '100':
                    # Train set 
                    #X_train = X_t
                    #y_train = y_t
                    X_train = np.append(X_train, X_t, axis=0)
                    y_train = np.append(y_train, y_t, axis=0)
                elif delayPktSize == '1400':
                    # Test set 
                    #X_test = X_t
                    #y_test = y_t
                    X_test = np.append(X_test, X_t, axis=0)
                    y_test = np.append(y_test, y_t, axis=0)
            print('debug9')
            print('---------------------------------------------------------')        
                    
                    
            
        # end of for over delayPktSize
    # end of for over measUeDevs
    
    #=============================================== Create the train and test set ==================================
    if classification:
            strat = y
    else: 
            strat = None
    #===================================
    if transfer_bw_dev:
        # train on samples from one dev and test on the other 
        # train test split
        # apparently subsampling within XGBoost does shuffling so no need to shuffle before sending in the data
        # For some reason it isnt doing so..so I am shuffling before sending it 
        # Random sampling of these samples is not doing anythig ? Why doesnt it affect the random selection of the XGB ? 
        print(np.shape(X))
        print(np.shape(y))
        data_temp = np.append(X, np.expand_dims(y, axis=1), axis=1)
        print(np.shape(data_temp))
        
        np.random.seed(IN_PARAM['rand_seed'])
        np.random.shuffle(data_temp)
        
        print(np.shape(data_temp))
        X = data_temp[:,:-1]
        y = data_temp[:,-1]
        print(np.shape(X))
        print(np.shape(y))
        
        ind = np.where(X_feats == 'ul_chipsetType')[0][0]
        X_train = X[ X[:,ind] == train_on_chipset, :]
        y_train = y[ X[:,ind] == train_on_chipset]
        
        X_test = X[ X[:,ind] != train_on_chipset, :]
        y_test = y[ X[:,ind] != train_on_chipset]
        
    elif test_on_one_chipset:
        # train test split
        X_train, X_test_tmp, y_train, y_test_tmp = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=IN_PARAM['rand_seed'], stratify=strat)    
        #X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=IN_PARAM['rand_seed'])
        ind = np.where(X_feats == 'ul_chipsetType')[0][0]
        X_test = X_test_tmp[ X_test_tmp[:,ind] == test_on_chipset, :]
        y_test = y_test_tmp[ X_test_tmp[:,ind] == test_on_chipset]
    elif train_test_on_diff_pkt_sizes:
        print('Nothing to do')
        # Do nothing train and test sets have already been prepared 
    else:
        # train test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=IN_PARAM['rand_seed'], stratify=strat)
        #X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, shuffle=True, random_state=IN_PARAM['rand_seed'])  
    
    print('data_train shape ' + str(X_train.shape))
    #print('data_val shape ' + str(X_val.shape))
    print('data_test shape ' + str(X_test.shape))

          
    #=============================================== Plot delay hist ================================== 
    # If classification it will just bin it
    if r == 0:
        plt.figure(1)
        plt.hist(y, bins=50, edgecolor='k')
        plt.xlabel('UL delay (ms)')
        plt.title('Histogram of all samples of UL delay')
        plt.show()
    
    #=============================================== Train and test the model ==================================
    
    # Model XGB
    OUT_PARAM = dict.fromkeys(OUT_PARAM, 0)
    model_save_num = model_save_num + 1
    start_time = time.time()
    
    model, history = evaluate_model(X_train, X_test, y_train, y_test, 
                                                                 model_save_path + str(IN_PARAM['model_save_num']) + '/', IN_PARAM, sample_weights=samp_weights)
    end_time = time.time()
    OUT_PARAM['runtime'] = end_time - start_time
    print('Time to train model: ', end_time - start_time)
    
    yhat_test = model.predict(X_test)
    yhat_train = model.predict(X_train)
    # what if the regression goes out of the class range
    
    if classification:
        # This actually does regression but with class labels, so it can be a bit confusing that I call this classification 
        if IN_PARAM['loss'] == 'class_mse':
            # convert the continuous regression outputs to class labels 
            yhat_train = np.round(yhat_train)
            yhat_train[yhat_train > (num_classes-1)] = num_classes-1
            yhat_train[yhat_train < 0] = 0
            yhat_test = np.round(yhat_test)
            yhat_test[yhat_test > (num_classes-1)] = num_classes-1
            yhat_test[yhat_test < 0] = 0
            
        
        # create a confusion matrix
        cf_matrix = confusion_matrix(y_test, yhat_test, normalize='true')
        sns.set(rc={'figure.figsize':(8,7)}, font_scale = 1.5)
        sns.heatmap(cf_matrix, annot=True, 
            fmt='.1%', cmap='Blues')
        
        # get accuracy, precision and recall
        OUT_PARAM['train_acc'] = accuracy_score(yhat_train, y_train)
        OUT_PARAM['train_prec'], OUT_PARAM['train_rec'], train_fscore, train_support = precision_recall_fscore_support(yhat_train, y_train, average='macro')
        OUT_PARAM['test_acc'] = accuracy_score(yhat_test, y_test)
        OUT_PARAM['test_prec'], OUT_PARAM['test_rec'], test_fscore, test_support = precision_recall_fscore_support(yhat_test, y_test, average='macro')
        avg_acc_over_runs = avg_acc_over_runs + OUT_PARAM['test_acc']
        avg_prec_over_runs = avg_prec_over_runs + OUT_PARAM['test_prec']
        avg_rec_over_runs = avg_rec_over_runs + OUT_PARAM['test_rec']
        #avg_baseline_err_over_runs = avg_baseline_err_over_runs + ???
        print(OUT_PARAM)
        print('ML model: ', IN_PARAM['eval_metric'],' (acc, prec, recall) for test set: ',  OUT_PARAM['test_acc'], OUT_PARAM['test_prec'], OUT_PARAM['test_rec'])
        #print('Baseline: ', IN_PARAM['eval_metric'], ' err for test set: ', compute_error(IN_PARAM['eval_metric'], np.repeat(baseline_pred, len(y_test)), y_test))
        
        # Also get the regression type performance metrics to capture the differeces in 
        # impact on misclassification between neighbouring and other classes 
        # convert labels to mean of the delay class for both true and pred
        #print('class_mae ', class_mae(yhat_test, y_test, num_classes, delay_class_edges))
        #baseline_class = np.digitize(np.repeat(baseline_pred, len(y_test)), delay_class_edges) - 1
        #print('baseline class_mae ', class_mae(baseline_class, y_test, num_classes, delay_class_edges))
        #avg_err_over_runs = avg_err_over_runs + class_mae(yhat_test, y_test, num_classes, delay_class_edges)
        #avg_baseline_err_over_runs = avg_baseline_err_over_runs + class_mae(baseline_class, y_test, num_classes, delay_class_edges)
        
    else: 
        OUT_PARAM['train_err'] = compute_error(IN_PARAM['eval_metric'], yhat_train, y_train)
        OUT_PARAM['test_err'] = compute_error(IN_PARAM['eval_metric'], yhat_test, y_test)
        avg_err_over_runs = avg_err_over_runs + OUT_PARAM['test_err']
        avg_baseline_err_over_runs = avg_baseline_err_over_runs + compute_error(IN_PARAM['eval_metric'], np.repeat(baseline_pred, len(y_test)), y_test)
        print(OUT_PARAM)
        print('ML model: ', IN_PARAM['eval_metric'],' err for test set: ', compute_error(IN_PARAM['eval_metric'], yhat_test, y_test))
        print('Baseline: ', IN_PARAM['eval_metric'], ' err for test set: ', compute_error(IN_PARAM['eval_metric'], np.repeat(baseline_pred, len(y_test)), y_test))
        print('Mean of the train set is: ', np.mean(y_train))
        print('Median of the train set is: ', np.median(y_train))

        # plot the cdf of the train error 
        ecdf_train = ECDF(yhat_train - y_train)
        plt.step(ecdf_train.x, ecdf_train.y)
        plt.axvline(x=0, color='red', linestyle='--')
        plt.axhline(y=ecdf_train(0), color='red', linestyle='--')
        plt.xlabel('Pred err (truth - pred)')
        plt.title('Train samples')
        #plt.hist((yhat_train - y_train), bins=200, edgecolor='k')
        #plt.xlim(-20, 50)
        plt.show()
        print('Train: Probability mass of pred err (truth-pred) below 0 is: ',  ecdf_train(0))
        print('Train: Probability mass of pred err (truth-pred) above 0 is: ',  1-ecdf_train(0))

        # plot the cdf of the test error 
        ecdf_test = ECDF(yhat_test - y_test)
        plt.step(ecdf_test.x, ecdf_test.y)
        plt.axvline(x=0, color='red', linestyle='--')
        plt.axhline(y=ecdf_test(0), color='red', linestyle='--')
        plt.xlabel('Pred err (truth - pred)')
        plt.title('Test samples')
        #plt.hist((yhat_test - y_test), bins=200, edgecolor='k')
        #plt.xlim(-20, 50)
        plt.show()
        print('Test: Probability mass of pred err (truth-pred) below 0 is: ',  ecdf_test(0))
        print('Test: Probability mass of pred err (truth-pred) above 0 is: ',  1-ecdf_test(0))

        #===================================== plot sorted samples of prediction overlayed with ground truth  ==========================
        # 
        if sort_test_samples:   
            train_baseline_vals = np.repeat(baseline_pred, len(y_train))
            test_baseline_vals = np.repeat(baseline_pred, len(y_test))

            tmp1 = np.append(np.expand_dims(y_train, axis=1), np.expand_dims(yhat_train, axis=1), axis=1)
            tmp1 = tmp1[tmp1[:, 0].argsort()]
            y_train = tmp1[:,0]
            yhat_train = tmp1[:,1]

            tmp2 = np.append(np.expand_dims(y_test, axis=1), np.expand_dims(yhat_test, axis=1), axis=1)
            tmp2 = tmp2[tmp2[:, 0].argsort()]
            y_test = tmp2[:,0]
            yhat_test = tmp2[:,1]
        
        #=============================================== bin the delay values to observe err per bin ==================================
        #
        # bin index for each delay value, so that we can put the values in the right bin 
        bin_indices = np.digitize(y_train, bin_edges)
        
        # I want to take all the delay values for each bin
        for bin_ind in np.unique(bin_indices):
            # these are the delay values in bin bin_edges[bin_ind]
            train_bin_uldelay_mean[bin_ind-1] = train_bin_uldelay_mean[bin_ind-1] + np.sum(y_train[bin_indices == bin_ind])
            train_bin_count[bin_ind-1] = train_bin_count[bin_ind-1] + len(y_train[bin_indices == bin_ind])
            
            # I want the corresponding err values for these delay values  
            train_bin_err_mean[bin_ind-1] = (train_bin_err_mean[bin_ind-1] + 
                                             np.sum(np.abs(y_train[bin_indices == bin_ind] - yhat_train[bin_indices == bin_ind]) ))
            train_bin_baseline_err_mean[bin_ind-1] = (train_bin_baseline_err_mean[bin_ind-1] + 
                                                     np.sum(np.abs(y_train[bin_indices == bin_ind] - train_baseline_vals[bin_indices == bin_ind]) ))
            train_bin_perc_err_mean[bin_ind-1] = (train_bin_perc_err_mean[bin_ind-1] + 
                                                  np.sum(np.abs((y_train[bin_indices == bin_ind] - yhat_train[bin_indices == bin_ind]))/(y_train[bin_indices == bin_ind]) ))
            train_bin_baseline_perc_err_mean[bin_ind-1] = (train_bin_baseline_perc_err_mean[bin_ind-1] + 
                                                          np.sum(np.abs((y_train[bin_indices == bin_ind] - train_baseline_vals[bin_indices == bin_ind]))/(y_train[bin_indices == bin_ind])) )
        
        # bin index for each delay value
        bin_indices = np.digitize(y_test, bin_edges)
        
        # I want to take all the delay values for each bin 
        for bin_ind in np.unique(bin_indices):
            # these are the delay values in bin bin_edges[bin_ind]
            test_bin_uldelay_mean[bin_ind-1] = test_bin_uldelay_mean[bin_ind-1] + np.sum(y_test[bin_indices == bin_ind])
            test_bin_count[bin_ind-1] = test_bin_count[bin_ind-1] + len(y_test[bin_indices == bin_ind])
            
            # I want the corresponding err values for these delay values
            test_bin_err_mean[bin_ind-1] = (test_bin_err_mean[bin_ind-1] + 
                                            np.sum(np.abs(y_test[bin_indices == bin_ind] - yhat_test[bin_indices == bin_ind])) )
            test_bin_baseline_err_mean[bin_ind-1] = (test_bin_baseline_err_mean[bin_ind-1] + 
                                                     np.sum(np.abs(y_test[bin_indices == bin_ind] - test_baseline_vals[bin_indices == bin_ind])))
            test_bin_perc_err_mean[bin_ind-1] = ( test_bin_perc_err_mean[bin_ind-1] + 
                                                 np.sum(np.abs((y_test[bin_indices == bin_ind] - yhat_test[bin_indices == bin_ind]))/(y_test[bin_indices == bin_ind])) )
            test_bin_baseline_perc_err_mean[bin_ind-1] = (test_bin_baseline_perc_err_mean[bin_ind-1] + 
                                                          np.sum(np.abs((y_test[bin_indices == bin_ind] - test_baseline_vals[bin_indices == bin_ind]))/(y_test[bin_indices == bin_ind])) ) 
    
        # Plot
        plot_y_yhat(y_train, yhat_train, y_test, yhat_test, model_save_path, IN_PARAM)
    
        # Convert the regression output to class labels and do confusion matrix
        cf_matrix = confusion_matrix(value_to_class_label(y_test, delay_class_edges), 
                                     value_to_class_label(yhat_test, delay_class_edges), normalize='true')
        sns.set(rc={'figure.figsize':(8,7)}, font_scale = 1.5)
        sns.heatmap(cf_matrix, annot=True, 
            fmt='.1%', cmap='Blues')
        
        
        #=============================================== plot q-q prediction versus ground truth  ==================================
        #
        plt.figure(figsize=(10, 10))
        plt.plot(y_test, yhat_test, 'b.')
        plt.plot([0,0], [delay_drop_th, delay_drop_th], 'k-')
        plt.xlabel('Test samples ground truth (ms)')
        plt.ylabel('Test samples prediction (ms)')
        plt.grid()
        plt.show()

        
    if not os.path.isdir(model_save_path):
        os.makedirs(model_save_path)
    #=============================================== plot feature importance ==================================
    # 
    # The length of importances reflects the number of features used
    importances = model.feature_importances_
    # increasing order in value and hence decreasing order in importance 
    # sort the importances and then fetch the index value of those importances 
    indices = np.argsort(importances)
    #This is in ascending order of 
    bar_vals = importances[np.flip(indices)[0:feat_filter]]
    bar_names = X_feats[np.flip(indices)[0:feat_filter]]
    #print(importances[np.flip(indices)[0:feat_filter]])
    print(X_feats[np.flip(indices)[0:feat_filter]])
    
    top_n_features = list( set(top_n_features).union(set(bar_names)))
    print('Top n feature list: ', top_n_features)
    plt.figure()
    plt.barh(range(len(bar_vals)), np.flip(bar_vals), color='b', align='center')
    plt.yticks(range(len(bar_vals)), np.flip(bar_names))
    
    plt.title('Feature importance XGB')
    plt.xlabel('Relative Importance')
    plt.savefig('feat_imp'+str(IN_PARAM['rand_seed'])+'.pdf', bbox_inches='tight')
    plt.show() 
        
print('')
print('')
print('===============================  DONE  ===================================')

In [ ]:
OUT_PARAM['test_prec'], OUT_PARAM['test_rec'], test_fscore, test_support = precision_recall_fscore_support(yhat_test, y_test, average='macro')
print('ML model: Macro (acc, prec, recall) for test set: ',  OUT_PARAM['test_acc'], OUT_PARAM['test_prec'], OUT_PARAM['test_rec'])
OUT_PARAM['test_prec'], OUT_PARAM['test_rec'], test_fscore, test_support = precision_recall_fscore_support(yhat_test, y_test, average='micro')
print('ML model: Micro (acc, prec, recall) for test set: ',  OUT_PARAM['test_acc'], OUT_PARAM['test_prec'], OUT_PARAM['test_rec'])
OUT_PARAM['test_prec'], OUT_PARAM['test_rec'], test_fscore, test_support = precision_recall_fscore_support(yhat_test, y_test, average='weighted')
print('ML model: Weighted (acc, prec, recall) for test set: ',  OUT_PARAM['test_acc'], OUT_PARAM['test_prec'], OUT_PARAM['test_rec'])



In [ ]:
if classification:
    print('Average: (acc, prec, recall) for test set over runs: ', avg_acc_over_runs, avg_prec_over_runs, avg_rec_over_runs)
else:    
    print('Average', IN_PARAM['eval_metric'], ' over the runs for ML model is: ', avg_err_over_runs/len(run_seeds))
    print('Average', IN_PARAM['eval_metric'], ' over the runs on the baseline is: ', avg_baseline_err_over_runs/len(run_seeds))
    print('-----------------------------------------------------------')
    print('Top n feature list size: ', len(top_n_features))
    print(top_n_features)
    print('-----------------------------------------------------------')

    print('Loss fun: ', IN_PARAM['loss'])
    print('Eval err fun: ', IN_PARAM['eval_metric'])
    # After going over all runs     
    fig, ax1 = plt.subplots(figsize=(16, 4))
    ax2 = ax1.twinx()
    ax1.plot(train_bin_uldelay_mean/train_bin_count, train_bin_err_mean/train_bin_count, 'r.-', label='XGB pred err (truth-pred)')
    #ax1.plot(train_bin_uldelay_mean/train_bin_count, train_bin_baseline_err_mean/train_bin_count, 'm.-', label='baseline pred err (truth-pred)')
    ax1.axhline(y=0, color='r', linestyle='--')
    ax1.set_ylabel('error')
    #ax1.set_ylim(-5,25)
    ax1.legend()
    plt.xlabel('uplink delay (ms)')
    plt.title('Train samples')
    plt.grid()
    ax2.set_ylabel('relative err')
    ax2.plot(train_bin_uldelay_mean/train_bin_count, train_bin_perc_err_mean/train_bin_count, 'g*-', label='XGB relative err')
    #ax2.plot(train_bin_uldelay_mean/train_bin_count, train_bin_baseline_perc_err_mean/train_bin_count, 'c*-', label='baseline relative err')
    ax2.axhline(y=0, color='g', linestyle='--')
    plt.xscale('log')
    ax1.legend(loc=6)
    ax2.legend(loc=1)
    plt.show() 

    plt.figure(figsize=(16, 2))
    plt.plot(train_bin_uldelay_mean/train_bin_count, train_bin_count, 'b*-')
    plt.xlabel('Train samples uplink delay bin (ms)')
    plt.xscale('log')
    plt.ylabel('bin count')
    plt.grid()
    plt.show()

    print('Loss fun: ', IN_PARAM['loss'])
    print('Eval err fun: ', IN_PARAM['eval_metric'])
    fig, ax1 = plt.subplots(figsize=(16, 4))
    ax2 = ax1.twinx()
    ax1.plot(test_bin_uldelay_mean/test_bin_count, test_bin_err_mean/test_bin_count, 'r.-', label='XGB pred err (truth-pred)')
    #ax1.plot(test_bin_uldelay_mean/test_bin_count, test_bin_baseline_err_mean/test_bin_count, 'm.-', label='baseline pred err (truth-pred)')
    ax1.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('uplink delay (ms)')
    ax1.set_ylabel('error')
    #ax1.set_ylim(-500,250)
    ax1.legend()
    plt.title('Test samples')
    plt.grid()
    ax2.set_ylabel('relative err')
    ax2.plot(test_bin_uldelay_mean/test_bin_count, test_bin_perc_err_mean/test_bin_count, 'g*-', label='relative err')
    #ax2.plot(test_bin_uldelay_mean/test_bin_count, test_bin_baseline_perc_err_mean/test_bin_count, 'c*-', label='baseline relative err')
    ax2.axhline(y=0, color='g', linestyle='--')
    plt.xscale('log')
    ax1.legend(loc=6)
    ax2.legend(loc=1)
    plt.show()

    plt.figure(figsize=(16, 2))
    plt.plot(test_bin_uldelay_mean/test_bin_count, test_bin_count, 'b*-')
    plt.xlabel('Test samples uplink delay bins (ms)')
    plt.ylabel('bin count')
    plt.xscale('log')
    plt.grid()
    plt.show()


In [ ]:
notebook_save_str = 'clas_using_loss_default_softprob'
os.system('cp train_and_eval_ML_models.ipynb '+'./saved_notebooks/'+notebook_save_str+'.ipynb')


In [ ]:
feat_union = ['ul_loadUe_weightBand', 'ul_loadUe_BSRestimate', 'ul_BSRestimateLcg6', 'ul_loadUe_deltaIcc', 'ul_nrOfCsiPart1Bits', 'ul_loadUe_RI', 'dl_pucchFormatType', 'ul_loadUe_BSValueLcg6', 'ul_rv', 'ul_tbSizeInBits', 'dl_loadUe_dciFormat', 'dl_loadUe_pucchResourceIndicator', 'nbm_loadUe_WBeamIndex', 'ul_loadUe_fda', 'dl_NBeamRsrpCurrent', 'dl_loadUe_transmissionAttempt', 'dl_loadUe_bbBearerRef7', 'ul_BSRestimateLcg4', 'dl_numOfPrbs', 'ul_loadUe_isTransformPrecoding', 'ul_loadUe_timingOffset', 'dl_numberOfSymbols', 'ul_BSValueLcg0', 'dl_tbSizeInBits', 'ul_loadUe_VBit', 'ul_loadUe_BSRestimateLcg2', 'ul_csiRequest', 'ul_loadUe_BSValueLcg4', 'dl_loadUe_weightBand', 'wbm_rsrp1', 'dl_loadUe_NBeamRsrpCurrent', 'ul_tpcCommand', 'ul_iccAchievable', 'ul_loadUe_preamblePwr', 'nbm_beam03', 'dl_drb5data', 'dl_pucchSfn', 'ul_loadUe_numOfLayers', 'nbm_rsrp01', 'dl_icc', 'ul_loadUe_iccAchievable', 'ul_fda', 'ul_loadUe_uciDecodedResult', 'ul_loadUe_BSValueLcg0', 'dl_dlCcIndex', 'nbm_loadUe_rsrp11', 'wbm_beam4', 'dl_loadUe_incrementalWeight', 'dl_loadUe_antennaPorts', 'ul_BSValueLcg5', 'ul_RI', 'dl_loadUe_fda', 'dl_loadUe_numBearers', 'ul_loadUe_csiRequest', 'dl_linkAdaptationUeMode', 'wbm_loadUe_cellId', 'ul_loadUe_ACK', 'dl_cellId', 'dl_bbBearerRef2', 'ul_loadUe_BSValueLcg7', 'dl_loadUe_cqi', 'dl_pdschHarqFeedbackTiming', 'dl_loadUe_pucchSfn', 'dl_loadUe_pdschHarqFeedbackTiming', 'ul_loadUe_isClpcSaturated', 'ul_loadUe_startSymbolPusch', 'dl_loadUe_dlCcIndex', 'dl_loadUe_drb7input', 'dl_bbBearerRef7', 'dl_loadUe_bbBearerRef2', 'ul_loadUe_beamIndex', 'dl_rank', 'dl_loadUe_drb4input', 'dl_numBearers', 'wbm_loadUe_beam2', 'nbm_loadUe_beam01', 'ul_loadUe_WBeamRsrpCurrent', 'ul_carrierAggregationUsed', 'nbm_loadUe_rsrp12', 'dl_drb4input', 'dl_mcsIndex', 'dl_sinr', 'dl_drb4data', 'ul_loadUe_harqProcessId', 'dl_transmissionAttempt', 'ul_loadUe_BSRestimateLcg7', 'dl_loadUe_isDrxEnabled', 'dl_drb0data', 'dl_loadUe_numOfPrbs', 'dl_loadUe_drb4data', 'ul_loadUe_numberOfSymbolsPusch', 'nbm_loadUe_ueTraceIdMsw', 'wbm_loadUe_beam3', 'dl_sectorIndex', 'dl_drb1input', 'dl_numberOfSrcBits', 'nbm_beam13', 'ul_clpcCarrierDemand', 'ul_wbUsedNbOverridden', 'ul_loadUe_puschTotalRxPsdAvg', 'dl_srOnPucch', 'dl_loadUe_sectorIndex', 'wbm_beam3', 'dl_drb0input', 'ul_BSRestimate', 'ul_BSRestimateLcg1', 'dl_loadUe_taValue', 'ul_maxRank', 'wbm_beam1', 'dl_loadUe_bbBearerRef3', 'dl_physicalCellId', 'ul_totalCellsReqScheduling', 'wbm_loadUe_beam4', 'nbm_loadUe_rsrp02', 'wbm_loadUe_WBeamIndexNewBest', 'dl_loadUe_WBeamRsrpCurrent', 'nbm_WBeamIndex', 'dl_loadUe_slot', 'dl_bbBearerRef1', 'dl_drb3data', 'nbm_loadUe_rsrp13', 'ul_macSduInBytes', 'dl_loadUe_srOnPucch', 'ul_measNumOfPrb', 'ul_transmissionAttempt', 'ul_loadUe_BSRestimateLcg3', 'dl_loadUe_deltaIcc', 'dl_dciFormat', 'ul_bbCellIndex', 'dl_loadUe_macCtrlElement', 'dl_weightBand', 'ul_loadUe_mcsIndex', 'ul_loadUe_antennaPorts', 'ul_timingOffset', 'dl_redundancyVersion', 'ul_loadUe_BSRestimateLcg1', 'ul_loadUe_wbUsedNbOverridden', 'dl_numberOfActivatedDlCells', 'dl_antennaPorts', 'dl_drb7input', 'dl_harqProcessId', 'nbm_loadUe_beam13', 'ul_loadUe_nrOfCsiPart1Bits', 'ul_dciFormat', 'dl_pucchSlotNo', 'nbm_NBeamIndexChosen', 'ul_precodingInfo', 'ul_loadUe_NBeamRsrpCurrent', 'ul_loadUe_harqFailure', 'dl_bbBearerRef4', 'dl_ndi', 'nbm_beam12', 'dl_loadUe_tda', 'nbm_loadUe_cellId', 'wbm_loadUe_WBeamIndexCurrent', 'ul_ndi', 'dl_loadUe_tbSizeInBits', 'wbm_cellId', 'dl_loadUe_icc', 'nbm_loadUe_beam02', 'dl_pucchResourceIndicator', 'ul_BSValueLcg2', 'ul_loadUe_macSduInBytes', 'ul_measNumOfLayers', 'dl_loadUe_drb1input', 'dl_WBeamRsrpCurrent', 'nbm_loadUe_noOfCriPerCsiReport', 'ul_antennaPorts', 'ul_loadUe_slot', 'wbm_beam2', 'ul_loadUe_startPrb', 'ul_slot', 'nbm_loadUe_rsrp01', 'ul_BSRestimateLcg0', 'ul_loadUe_numOfPrbs', 'ul_isDrxEnabled', 'ul_loadUe_BSRestimateLcg0', 'wbm_rsrp4', 'ul_BSValueLcg4', 'dl_loadUe_pucchSlotNo', 'ul_preamblePwr', 'wbm_loadUe_beam1', 'dl_ACK', 'ul_isPrimaryCell', 'dl_tda', 'ul_loadUe_postEqSinr0', 'dl_loadUe_sinr', 'ul_loadUe_pCmaxCIndex', 'ul_loadUe_cellId', 'dl_loadUe_physicalCellId', 'dl_loadUe_beamIndex', 'dl_loadUe_numberOfSymbols', 'dl_loadUe_drb2data', 'dl_loadUe_drb1data', 'ul_BSRestimateLcg7', 'ul_numOfLayers', 'ul_loadUe_numberOfActivatedUlCells', 'ul_loadUe_BSValueLcg1', 'nbm_beam11', 'ul_loadUe_BSValueLcg5', 'dl_loadUe_newData/reTx(1/0)', 'dl_loadUe_bbBearerRef6', 'ul_loadUe_bbCellIndex', 'nbm_loadUe_NBeamIndexChosen', 'dl_loadUe_numberOfActivatedDlCells', 'ul_isClpcSaturated', 'ul_loadUe_BSRestimateLcg5', 'ul_loadUe_tbSizeInBits', 'ul_BSValueLcg1', 'ul_loadUe_maxRank', 'dl_fda', 'ul_DTX', 'ul_loadUe_clpcCarrierDemand', 'dl_cqi', 'dl_dai', 'wbm_rsrp3', 'ul_numOfPrbs', 'dl_drb3input', 'ul_puschTotalRxPsdAvg', 'nbm_loadUe_NBeamIndexCurrent', 'nbm_cellId', 'ul_loadUe_postEqSinr1', 'dl_loadUe_drb3input', 'ul_pCmaxCIndex', 'ul_numberOfSymbolsPusch', 'dl_loadUe_mcsIndex', 'dl_newData/reTx(1/0)', 'ul_weightBand', 'nbm_rsrp02', 'ul_uciDecodedResult', 'ul_loadUe_BSValueLcg2', 'ul_powerHeadRoomIndex', 'ul_loadUe_rv', 'ul_incrementalWeight', 'dl_loadUe_drb2input', 'dl_loadUe_drb7data', 'dl_drb6input', 'wbm_rsrp2', 'dl_loadUe_bbCellIndex', 'dl_bbBearerRef3', 'dl_loadUe_feedbackIndex', 'ul_startPrb', 'ul_cellId', 'wbm_loadUe_rsrp1', 'dl_loadUe_rank', 'ul_ACK', 'ul_beamIndex', 'dl_isSrBitIncluded', 'ul_numberOfActivatedUlCells', 'dl_bbBearerRef6', 'ul_loadUe_transmissionAttempt', 'dl_loadUe_isPrimaryCell', 'wbm_loadUe_rsrp4', 'ul_BSValueLcg3', 'dl_macCtrlElement', 'dl_drb6data', 'dl_isPrimaryCell', 'ul_harqFailure', 'dl_bbCellIndex', 'ul_loadUe_totalCellsReqScheduling', 'ul_postEqSinr0', 'ul_loadUe_carrierAggregationUsed', 'dl_loadUe_bbBearerRef4', 'ul_loadUe_powerHeadRoomIndex', 'ul_BSValueLcg6', 'nbm_loadUe_beam03', 'dl_drb2data', 'nbm_rsrp13', 'dl_isDrxEnabled', 'ul_loadUe_measNumOfPrb', 'dl_feedbackIndex', 'ul_loadUe_isDrxEnabled', 'dl_loadUe_bbBearerRef1', 'ul_loadUe_ulRequestTypeBitmap', 'nbm_rsrp03', 'dl_slot', 'dl_loadUe_drb5input', 'ul_loadUe_BSRestimateLcg4', 'dl_beamIndex', 'dl_loadUe_isSrBitIncluded', 'ul_BSRestimateLcg3', 'ul_WBeamRsrpCurrent', 'dl_loadUe_DTX', 'nbm_rsrp11', 'ul_BSRestimateLcg5', 'dl_loadUe_redundancyVersion', 'ul_ulRequestTypeBitmap', 'wbm_WBeamIndexNewBest', 'ul_ulschIndicator', 'dl_drb1data', 'dl_drb2input', 'ul_loadUe_BSRestimateLcg6', 'nbm_loadUe_rsrp03', 'dl_loadUe_wbUsedNbOverridden', 'dl_loadUe_drb6input', 'nbm_loadUe_beam11', 'nbm_beam01', 'dl_loadUe_linkAdaptationUeMode', 'ul_loadUe_linkAdaptationUeMode', 'dl_loadUe_bbBearerRef0', 'dl_drb5input', 'dl_loadUe_drb6data', 'ul_loadUe_BSValueLcg3', 'dl_loadUe_drb0data', 'ul_loadUe_sinrAchievable', 'nbm_loadUe_beam12', 'ul_loadUe_incrementalWeight', 'dl_loadUe_dai', 'dl_loadUe_ueTraceIdMsw', 'ul_loadUe_ndi', 'ul_startSymbolPusch', 'dl_loadUe_cellId', 'ul_isTransformPrecoding', 'dl_loadUe_drb5data', 'dl_loadUe_drb3data', 'ul_deltaIcc', 'nbm_noOfCriPerCsiReport', 'dl_loadUe_drb0input', 'ul_mcsIndex', 'ul_chipsetType', 'dl_loadUe_numberOfSrcBits', 'nbm_beam02', 'ul_postEqSinr1', 'ul_loadUe_isPrimaryCell', 'dl_incrementalWeight', 'dl_loadUe_ACK', 'ul_BSRestimateLcg2', 'ul_NBeamRsrpCurrent', 'dl_DTX', 'dl_bbBearerRef5', 'dl_loadUe_bbBearerRef5', 'ul_harqProcessId', 'dl_loadUe_pucchFormatType', 'wbm_loadUe_rsrp2', 'dl_taValue', 'ul_loadUe_precodingInfo', 'ul_BSValueLcg7', 'wbm_loadUe_ueTraceIdMsw', 'dl_loadUe_ndi', 'dl_deltaIcc', 'dl_drb7data', 'ul_loadUe_DTX', 'ul_linkAdaptationUeMode', 'wbm_loadUe_rsrp3', 'nbm_NBeamIndexCurrent', 'dl_loadUe_harqProcessId', 'ul_loadUe_ulschIndicator', 'dl_wbUsedNbOverridden', 'ul_loadUe_measNumOfLayers', 'nbm_rsrp12', 'dl_bbBearerRef0', 'wbm_WBeamIndexCurrent', 'ul_sinrAchievable']
print(feat_union)
print(len(feat_union))

In [ ]:
# List of featurtes I think will not generalize 
filtered_cols = [i for i in feat_union if 'eamIndex' in i]
print(filtered_cols)